# Word Embeddings

In [1]:
import os
import pandas as pd
import numpy as np

clean_news_df = pd.read_csv(os.path.join(os.getcwd(),"dataset\\clean_news_df.csv"))

In [2]:
clean_news_df

,true_or_fake,text,cleaned_text
0,true,"As U.S. budget fight looms, Republicans flip t...",budget fight loom republican flip fiscal scrip...
1,true,U.S. military to accept transgender recruits o...,military accept transgender recruit monday pen...
2,true,Senior U.S. Republican senator: 'Let Mr. Muell...,senior republican senator let mueller job wash...
3,true,FBI Russia probe helped by Australian diplomat...,fbi russia probe help australian diplomat tip ...
4,true,Trump wants Postal Service to charge 'much mor...,trump want postal service charge much amazon s...
...,...,...,...
44893,fake,McPain: John McCain Furious That Iran Treated ...,mcpain john mccain furious iran treat sailor w...
44894,fake,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,justice yahoo settle mail privacy class action...
44895,fake,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,sunnistan ally safe zone plan take territorial...
44896,fake,How to Blow $700 Million: Al Jazeera America F...,blow million jazeera america finally call quit...


In [3]:
# remove empty values
clean_news_df.dropna(inplace=True)

In [4]:
# split into training set (test set?) and validation set

# train = 80, test = 20
# random_seed = 42

from sklearn.model_selection import train_test_split

X = clean_news_df['cleaned_text'].str.split()
y = clean_news_df['true_or_fake']

X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle=True,random_state=42,test_size=0.2,stratify=y)

X_train = X_train.reset_index(drop = True)
X_test = X_test.reset_index(drop = True)


In [5]:
# train a Word2Vec model

from gensim.models import Word2Vec
w2v_model = Word2Vec(X_train, vector_size=200, window=5, min_count=1)

In [6]:
vocab=list(w2v_model.wv.key_to_index.keys())
print(len(vocab))

87044


In [7]:
vocab

['say',
 'trump',
 'state',
 'president',
 'would',
 'people',
 'year',
 'make',
 'one',
 'republican',
 'new',
 'take',
 'obama',
 'clinton',
 'also',
 'house',
 'government',
 'time',
 'reuters',
 'tell',
 'donald',
 'get',
 'call',
 'election',
 'country',
 'american',
 'right',
 'white',
 'could',
 'party',
 'campaign',
 'like',
 'vote',
 'two',
 'official',
 'know',
 'come',
 'news',
 'last',
 'report',
 'united',
 'use',
 'work',
 'include',
 'first',
 'want',
 'group',
 'law',
 'back',
 'even',
 'hillary',
 'washington',
 'see',
 'day',
 'video',
 'show',
 'former',
 'support',
 'give',
 'think',
 'court',
 'week',
 'medium',
 'security',
 'many',
 'woman',
 'attack',
 'national',
 'plan',
 'may',
 'bill',
 'police',
 'well',
 'political',
 'leader',
 'need',
 'million',
 'democrat',
 'russia',
 'image',
 'accord',
 'ask',
 'way',
 'since',
 'administration',
 'month',
 'percent',
 'tax',
 'leave',
 'issue',
 'america',
 'presidential',
 'senate',
 'member',
 'force',
 'democrat

In [8]:
words = set(w2v_model.wv.index_to_key)
X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train])
X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_test])
X_train_avg = []
for v in X_train_vect:
        X_train_avg.append(v.mean(axis=0))

X_test_avg = []
for v in X_test_vect:
        X_test_avg.append(v.mean(axis=0))

C:\Users\desmo\AppData\Local\Temp\ipykernel_13272\2131801328.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_train])
C:\Users\desmo\AppData\Local\Temp\ipykernel_13272\2131801328.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_vect = np.array([np.array([w2v_model.wv[i] for i in ls if i in words]) for ls in X_test])


# Model Building

## Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train_avg, y_train)
lr.score(X_test_avg,y_test)


c:\Users\desmo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9829583426152818

In [10]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score, classification_report

lr_preds = lr.predict(X_test_avg)
print("Accuracy: ", accuracy_score(y_true=y_test, y_pred=lr_preds))
print("Precision: ", precision_score(y_true=y_test, y_pred=lr_preds, pos_label='true'))
print("Recall: ", recall_score(y_true=y_test, y_pred=lr_preds, pos_label='true'))
print("F1-Score: ", f1_score(y_true=y_test, y_pred=lr_preds, pos_label='true'))
print(classification_report(y_test, lr_preds))

print("Confusion Matrix: \n", confusion_matrix(y_true=y_test, y_pred=lr_preds).ravel())

Accuracy:  0.9829583426152818
Precision:  0.9793407613741876
Recall:  0.985057202895167
F1-Score:  0.9821906646490514
              precision    recall  f1-score   support

        fake       0.99      0.98      0.98      4695
        true       0.98      0.99      0.98      4283

    accuracy                           0.98      8978
   macro avg       0.98      0.98      0.98      8978
weighted avg       0.98      0.98      0.98      8978

Confusion Matrix: 
 [4606   89   64 4219]


In [11]:
lr.fit(X_train_avg, y_train)

c:\Users\desmo\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


In [13]:

param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # Regularization strength parameter
    'solver': ['liblinear', 'saga']  # Solvers to consider
}


In [14]:

grid_search = GridSearchCV(estimator=LogisticRegression(max_iter=1000), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)


In [15]:

grid_search.fit(X_train_avg, y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'solver': ['liblinear', 'saga']},
             verbose=2)

In [16]:

print("Best hyperparameters:", grid_search.best_params_)


Best hyperparameters: {'C': 10, 'solver': 'saga'}


In [17]:

bestlr = grid_search.best_estimator_


In [18]:

best_lr_score = bestlr.score(X_test_avg, y_test)
print("Best model accuracy:", best_lr_score)


Best model accuracy: 0.9837380262864781


In [19]:

best_lr_preds = bestlr.predict(X_test_avg)


In [20]:
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,f1_score, classification_report

print("Accuracy: ", accuracy_score(y_true=y_test, y_pred=best_lr_preds))
print("Precision: ", precision_score(y_true=y_test, y_pred=best_lr_preds, pos_label='true'))
print("Recall: ", recall_score(y_true=y_test, y_pred=best_lr_preds, pos_label='true'))
print("F1-Score: ", f1_score(y_true=y_test, y_pred=best_lr_preds, pos_label='true'))
print(classification_report(y_test, best_lr_preds))

print("Confusion Matrix: \n", confusion_matrix(y_true=y_test, y_pred=best_lr_preds).ravel())

Accuracy:  0.9837380262864781
Precision:  0.9798190675017397
Recall:  0.986224608918982
F1-Score:  0.9830114033046312
              precision    recall  f1-score   support

        fake       0.99      0.98      0.98      4695
        true       0.98      0.99      0.98      4283

    accuracy                           0.98      8978
   macro avg       0.98      0.98      0.98      8978
weighted avg       0.98      0.98      0.98      8978

Confusion Matrix: 
 [4608   87   59 4224]


## Naive Bayes

## Random Forest

## Gradient Boosting

### Hyperparameter Tuning